### Webscrapping with Pandas pd.read_html()  + ODBC on SQL Server with pyodbc

#### Source : Wikipedia / Liste des pays par altitude moyenne (l'altitude moyenne mondiale est de 840 mètres)

Liste des pays et territoires à statut spécial par altitude moyenne. 

In [56]:
import numpy as np
import pandas as pd
# Beautifulsoup already installed into Anaconda environment

# Librairy PyODBC for SQL Server connection
import os
import pyodbc
# setup SQL Server connection session
server = '-PCSJN\DATAVIZ'  
database = 'Climbing_DW_BI'
connection_string= (
                    'DRIVER={SQL Server};SERVER='
                     + server
                     + ';DATABASE='
                     + database 
                     + ';Trusted_Connection=yes'                  
                    )

# Function for injecting dataframe content into SQL Server Table
def template_SQL_insert_values(table_name,dict):
    
    columns = "("
    for k in dict.keys():
        columns += str(k) + ','
    columns = columns[:-1] ### remove last comma
    columns += ")"
    
    req = f"""INSERT INTO {table_name} {columns} VALUES {tuple(dict.values())}"""
    return req

In [57]:
### 1er url
Wiki_Countries_Mean_Altitude = "https://fr.wikipedia.org/wiki/Liste_des_pays_par_altitude_moyenne"
### récuperer toutes les tables html vu sur le site (url)
Dataset_Wiki_Countries_Mean_Altitude = pd.read_html(Wiki_Countries_Mean_Altitude)
type(Dataset_Wiki_Countries_Mean_Altitude), len(Dataset_Wiki_Countries_Mean_Altitude)

(list, 4)

In [58]:
Dataset_Wiki_Countries_Mean_Altitude[0]

,Rang,Pays,Altitude
0,1,Tadjikistan,3186
1,2,Kirghizistan,2988
2,3,Népal,2565
3,4,Bhoutan,2220
4,5,Lesotho,2161
...,...,...,...
158,158,Gambie,34
159,159,Pays-Bas,30
160,160,Qatar,28
161,161,Singapour,15


In [59]:
Dataset_Wiki_Countries_Mean_Altitude[0].columns = ['Rang','Nation','Altitude']
Dataset_Wiki_Countries_Mean_Altitude[0]

,Rang,Nation,Altitude
0,1,Tadjikistan,3186
1,2,Kirghizistan,2988
2,3,Népal,2565
3,4,Bhoutan,2220
4,5,Lesotho,2161
...,...,...,...
158,158,Gambie,34
159,159,Pays-Bas,30
160,160,Qatar,28
161,161,Singapour,15


In [60]:
# retrouver par tatonnement Invalid column name 'Côte d'Ivoire' pour script d'insertion de data ds SQL server
Dataset_Wiki_Countries_Mean_Altitude[0].loc[124,:]

Rang                  124
Nation      Côte d'Ivoire
Altitude              250
Name: 124, dtype: object

In [61]:
# Remplacement 'Côte d'Ivoire'
Dataset_Wiki_Countries_Mean_Altitude[0].iloc[124,1] = 'Cote d Ivoire'
Dataset_Wiki_Countries_Mean_Altitude[0].loc[124,:]

Rang                  124
Nation      Cote d Ivoire
Altitude              250
Name: 124, dtype: object

In [62]:
Dataset_Wiki_Countries_Mean_Altitude[0].dtypes

Rang         int64
Nation      object
Altitude    object
dtype: object

In [71]:
Wiki_Geography_perCountries_Mean_Altitude = Dataset_Wiki_Countries_Mean_Altitude[0]

# script de création d'une table ds SQL 
create_table = """Create TABLE Wiki_Geography_perCountries_Mean_Altitude
                                   ( 
                                        id INT IDENTITY PRIMARY KEY,
                                        Rang INT,
                                        Nation VARCHAR(200),
                                        Altitude VARCHAR(50),                                                                                                                                              
                                   ) """

# launch session
cnxn = pyodbc.connect(connection_string,autocommit=True) # no cursor.commit as Autocommit already
cursor = cnxn.cursor() 

try: 
    # load script "create " 
    cursor.execute(create_table) 
    
    # population de la table ds SQL
    for i,row in Wiki_Geography_perCountries_Mean_Altitude.iterrows(): 
        row_dict=row.to_dict()
        temp = template_SQL_insert_values('Wiki_Geography_perCountries_Mean_Altitude',row_dict)
        cursor.execute(temp)
   
except Exception as err: 
    print('err:',err) 
try: 
    cursor.close()  # close session
except Exception as err: 
    print('err:',err)

In [64]:
Dataset_Wiki_Countries_Mean_Altitude[1]

,Rang,Pays,Pays reconnu,Altitude
0,NaN,Sahara occidental,Maroc,256


In [65]:
Df_1_Wiki_Occidental_Sahara_Mean_Altitude = Dataset_Wiki_Countries_Mean_Altitude[1]

In [66]:
Dataset_Wiki_Countries_Mean_Altitude[2]

,Rang,Pays ou territoire,Pays responsable,Altitude
0,NaN,Antarctique,Aucun,2300
1,NaN,Groenland,Danemark,1792
2,NaN,Porto Rico,États-Unis,263
3,NaN,Guyane,France,168


In [67]:
Df_2_Wiki_Territories_Mean_Altitude = Dataset_Wiki_Countries_Mean_Altitude[2]

In [68]:
Dataset_Wiki_Countries_Mean_Altitude[3]

,v · mListes de pays,v · mListes de pays.1
0,Démographie,Population 1700 1800 1907 Densité de populatio...
1,Économie,Monnaies Dette publique Dette extérieure Égali...
2,Éducation,Dépenses d'éducation Taux d'alphabétisation Cl...
3,Environnement,Performance environnementale Empreinte écologi...
4,Géographie,Continent Superficie forestière Sans cours d'e...
5,Militaire,Paix Dépenses militaires Forces militaires Ne ...
6,Politique,États membres de l'ONU Date d'acquisition de l...
7,Religion,Par religion Irréligion Bouddhisme Christianis...
8,Santé,Dépenses de santé Consommation de médicaments ...
9,Droit,Peine de mort exécutions les plus récentes Tau...
